# Balancer Connectivity Analysis

In [10]:
import os
import pandas as pd
import plotly.express as px
from google.cloud import bigquery
from decimal import Decimal

pd.set_option('max_colwidth', 70)
os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = '/Users/fabiomendes/Downloads/balancer-314012-de7efb77a81d.json'

In [77]:
FROM_BLOCK_TIMESTAMP = "'2020-11-02 00:00:00'"
TO_BLOCK_TIMESTAMP = "'2020-12-20 23:59:59'" 

MY_TIMEOUT = 300

client = bigquery.Client()

## 1. Querying Uniswap ETH/DAI trades

In [102]:
uni_routers = {
    '0x7a250d5630B4cF539739dF2C5dAcb4c659F2488D'.lower(): 'Router'
}

uni_contracts = {
    '0xa478c2975ab1ea89e8196811f51a7b7ade33eb11'.lower(): 'DAI'
}

In [103]:
query = """
    SELECT 
        swaps.block_number,
        swaps.block_timestamp,
        swaps.transaction_hash,
        swaps.log_index AS transaction_index,
        swaps.sender AS transaction_sender,
        swaps.amount0In,
        swaps.amount1In,
        swaps.amount0Out,
        swaps.amount1Out,
        txns.receipt_gas_used AS gas_used,
        txns.gas_price
    FROM 
        `blockchain-etl.ethereum_uniswap.UniswapV2Pair_event_Swap` AS swaps
    INNER JOIN
        `bigquery-public-data.crypto_ethereum.transactions` AS txns
    ON
      swaps.transaction_hash = txns.hash
    WHERE TRUE
      AND swaps.to IN ({router_list})
      AND swaps.contract_address IN ({contract_list})
      AND swaps.block_timestamp >= {from_block_ts} 
      AND swaps.block_timestamp <= {to_block_ts}
      AND txns.block_timestamp >= {from_block_ts} 
      AND txns.block_timestamp <= {to_block_ts}
""".format(
    router_address=ROUTER_ADDRESS,
    from_block_ts=FROM_BLOCK_TIMESTAMP, 
    to_block_ts=TO_BLOCK_TIMESTAMP,
    router_list=(','.join(["'{}'".format(k) for k in list(uni_routers.keys())])),
    contract_list=(','.join(["'{}'".format(k) for k in list(uni_contracts.keys())])))

query_job = client.query(query)
iterator = query_job.result(timeout=MY_TIMEOUT)
rows = list(iterator)
uniswap = pd.DataFrame(data=[list(x.values()) for x in rows], columns=list(rows[0].keys()))
uniswap.head()

,block_number,block_timestamp,transaction_hash,transaction_index,transaction_sender,amount0In,amount1In,amount0Out,amount1Out,gas_used,gas_price
0,11265665,2020-11-16 00:25:54+00:00,0x99589005e47313364af40acd5a18fbee21aea360f6055a75b66bd4134a441bc8,126,0x7a250d5630b4cf539739df2c5dacb4c659f2488d,150603962564358545841,0,0,337716697434208624,171844,30000000000
1,11269199,2020-11-16 13:41:07+00:00,0x7b73a67f3c3d6fdd44e0e44b3cfe1bb7b9ed36c7c264d80689dbd32e0389537b,206,0x7a250d5630b4cf539739df2c5dacb4c659f2488d,1570464946983619764722,0,0,3472833570032821671,177298,38720000000
2,11267926,2020-11-16 08:49:04+00:00,0x027e77c4a76634f403250918cf15f92982e96ed12da631bc7f12cf2f2760283f,143,0x7a250d5630b4cf539739df2c5dacb4c659f2488d,2066272253953350861013,0,0,4557902872180071939,177298,32000001459
3,11270277,2020-11-16 17:35:15+00:00,0x53567ecebe64747d67369e970cca005a4892bae46adcd36d384db21eb3517144,29,0x7a250d5630b4cf539739df2c5dacb4c659f2488d,18064449097806366485238,0,0,39091434962120423835,890437,90000000000
4,11269899,2020-11-16 16:12:56+00:00,0x71b3531f6be91699ed3cc260b3f167c29a3d1e224da790c6da34bc8c3e544468,347,0x7a250d5630b4cf539739df2c5dacb4c659f2488d,16718737860000000000,0,0,36515212635449673,94223,56000000000


In [104]:
uniswap = uniswap.sort_values(['block_number','transaction_index']).reset_index(drop=True)
uniswap.drop_duplicates(subset='transaction_hash', keep=False, inplace=True)

uniswap['amount0In'] = uniswap['amount0In'].apply(Decimal)
uniswap['amount1In'] = uniswap['amount1In'].apply(Decimal)
uniswap['amount0Out'] = uniswap['amount0Out'].apply(Decimal)
uniswap['amount1Out'] = uniswap['amount1Out'].apply(Decimal)

In [105]:
uniswap['token_in'] = uniswap['amount0In'].apply(lambda amount0In: 'DAI' if amount0In > 0 else 'ETH')
uniswap['token_out'] = uniswap['amount0Out'].apply(lambda amount0Out: 'DAI' if amount0Out > 0 else 'ETH')

uniswap['amount_in'] = uniswap['amount0In'] + uniswap['amount1In']
uniswap['amount_out'] = uniswap['amount0Out'] + uniswap['amount1Out']

uniswap['transaction_fee'] = uniswap['gas_used'] * uniswap['gas_price']

In [106]:
uniswap.drop(['amount0In', 'amount1In', 'amount0Out', 'amount1Out'], axis=1, inplace=True)

In [107]:
uniswap

,block_number,block_timestamp,transaction_hash,transaction_index,transaction_sender,gas_used,gas_price,token_in,token_out,amount_in,amount_out,transaction_fee
0,11174331,2020-11-02 00:00:58+00:00,0xc8d0d9abfcc80cbd5973c5c378d1160a85ff1e591496a1d0f3b5d4d321279e24,52,0x7a250d5630b4cf539739df2c5dacb4c659f2488d,109247,100500000000,DAI,ETH,22756348999999997083648,57978263943916472260,10979323500000000
1,11174335,2020-11-02 00:01:49+00:00,0xe00fdb07eca1402994be16cb246cd6e6521603696d72c6fe68b4d74c0d51507c,293,0x7a250d5630b4cf539739df2c5dacb4c659f2488d,109242,33137500000,DAI,ETH,49384474513737528836212,125728500000000000000,3620006775000000
2,11174338,2020-11-02 00:02:24+00:00,0x0312f5c9fdff0e6d4c5bcfa094dc8d5376ffdadd79861df6918d4b0d66190f58,191,0x7a250d5630b4cf539739df2c5dacb4c659f2488d,109242,34375000000,DAI,ETH,98743436648724119725195,251129300000000016384,3755193750000000
3,11174342,2020-11-02 00:03:32+00:00,0x1011a29b9d1f32e1c7fb390ef3aa990deba018c8367ad946259e2c2fdf3c1c65,335,0x7a250d5630b4cf539739df2c5dacb4c659f2488d,175233,24200000000,DAI,ETH,822119303770518643856,2088720070747335950,4240638600000000
4,11174356,2020-11-02 00:05:22+00:00,0xa078dcf23beb9382dcb695939b27c848b4f66ed95c3b5c1e4d9b5a51b100201f,391,0x7a250d5630b4cf539739df2c5dacb4c659f2488d,106239,24200000000,DAI,ETH,10001358979888216018,25406845000269931,2570983800000000
...,...,...,...,...,...,...,...,...,...,...,...,...
44061,11493361,2020-12-20 23:58:33+00:00,0x5cf7dfa2374a7eeb9b824854d5b456a8111892cfc65cd6d1101ffbee54ecc233,215,0x7a250d5630b4cf539739df2c5dacb4c659f2488d,94259,26620000000,DAI,ETH,448327905535609297181,700721545279800473,2509174580000000
44062,11493361,2020-12-20 23:58:33+00:00,0x14071d934611b75c6832d442f40bb047a3df5f6263d075a4b776ca99fb04d1a1,220,0x7a250d5630b4cf539739df2c5dacb4c659f2488d,109235,26620000000,DAI,ETH,450000000000000000000,703324940189422865,2907835700000000
44063,11493363,2020-12-20 23:59:06+00:00,0xc01b251a028b191f8bf20619f06014f81d9e5923a11ecb357e315914324d2ebc,282,0x7a250d5630b4cf539739df2c5dacb4c659f2488d,189502,31910000000,DAI,ETH,17028044467908107295354,26605464969391090195,6047008820000000
44064,11493363,2020-12-20 23:59:06+00:00,0x8cd8d3a025e95c9c7b17747642cf35b9365d088b62ebcaeb660679b898084d34,362,0x7a250d5630b4cf539739df2c5dacb4c659f2488d,150632,30000000000,DAI,ETH,1023457818872548525218,1598642638364813229,4518960000000000


In [91]:
fig = px.histogram(uniswap, x='token_out')
fig.show()

In [7]:
def calc_dai_price(row):
    if (row.dai_amount_out != 0) & (row.eth_amount_in != 0):
        return row.dai_amount_out / row.eth_amount_in
    else:
        return row.dai_amount_in / row.eth_amount_out

def calc_eth_price(row):
    if (row.eth_amount_out !=0) & (row.dai_amount_in != 0):
        return row.eth_amount_out / row.dai_amount_in
    else:
        return row.eth_amount_in / row.dai_amount_out

In [8]:
#uniswap['dai_price'] = uniswap.apply(lambda row: calc_dai_price(row), axis=1)
#uniswap['eth_price'] = uniswap.apply(lambda row: calc_eth_price(row), axis=1)

uniswap['dai_amount_out'][uniswap['dai_amount_out'] != 0] / uniswap['eth_amount_in'][uniswap['eth_amount_in'] != 0] 
uniswap['dai_price'] = uniswap['dai_amount_in'][uniswap['dai_amount_in'] != 0] / uniswap['eth_amount_out'][uniswap['eth_amount_out'] != 0] 
uniswap['eth_price'] = uniswap['eth_amount_out'][uniswap['eth_amount_out'] != 0] / uniswap['dai_amount_in'][uniswap['dai_amount_in'] != 0]
uniswap['eth_price'] = uniswap['eth_amount_in'][uniswap['eth_amount_in'] != 0] / uniswap['dai_amount_out'][uniswap['dai_amount_out'] != 0]
uniswap

,block_timestamp,block_number,transaction_hash,log_index,contract_address,sender,dai_amount_in,eth_amount_in,dai_amount_out,eth_amount_out,to,dai_price,eth_price
0,2020-11-18 17:28:57+00:00,11283273,0x033eba188abdff3fd4b5de6c292c427727b729933c4c41bc48b26dfd43ee46b4,39,0xa478c2975ab1ea89e8196811f51a7b7ade33eb11,0x7a250d5630b4cf539739df2c5dacb4c659f2488d,0,178373339036740659,84000000000000000000,0,0xc47dec7ffde829043b91e904fdaf1e048bdd482c,NaN,0.00212349213138976975
1,2020-11-18 17:28:57+00:00,11283273,0x2024c774e832ac2b059c5bacc3f3736d8e50f5997658daed17ab4bb7dcad5c19,61,0xa478c2975ab1ea89e8196811f51a7b7ade33eb11,0x7a250d5630b4cf539739df2c5dacb4c659f2488d,897587599446926563429,0,0,1894585560969778077,0x7a250d5630b4cf539739df2c5dacb4c659f2488d,473.7646153006043315626955134,NaN
2,2020-11-18 17:28:57+00:00,11283273,0x1c90931720f48b5cd3cea5a1a6c64423dd812a5f3ca79a0875b6b59133708d72,261,0xa478c2975ab1ea89e8196811f51a7b7ade33eb11,0x7a250d5630b4cf539739df2c5dacb4c659f2488d,899033824865797806287,0,0,1897603438171453174,0x1107b6081231d7f256269ad014bf92e041cb08df,473.7732904468778011791623465,NaN
3,2020-11-18 17:28:58+00:00,11283274,0x5a89cc2850fabbda425bd025966be5d470ed1cdb3f4a56bbb34761df53f16d0b,177,0xa478c2975ab1ea89e8196811f51a7b7ade33eb11,0x7a250d5630b4cf539739df2c5dacb4c659f2488d,0,1000000000000000000,470936972491528376645,0,0x427e5697747b64097864bc681d59ae4aab3dccb6,NaN,0.002123426399735452643146684404
4,2020-11-18 17:29:08+00:00,11283275,0x729e1164d74690afc651a874f5d368152bd756522200e69d7f985becd9af0e83,140,0xa478c2975ab1ea89e8196811f51a7b7ade33eb11,0x7a250d5630b4cf539739df2c5dacb4c659f2488d,0,226853230566684541,106832942022602339465,0,0xf00e80f0de9aea0b33aa229a4014572777e422ee,NaN,0.002123438953115134215329270977
...,...,...,...,...,...,...,...,...,...,...,...,...,...
631153,2021-05-24 20:56:18+00:00,12499334,0xb2e2cda3cb14b8b0fdbdedb9e5ee2b901f767e20c85b17ffa814e8945f7c1eb6,166,0xa478c2975ab1ea89e8196811f51a7b7ade33eb11,0x7a250d5630b4cf539739df2c5dacb4c659f2488d,0,3531749261365351777,9193235653459291292058,0,0x3ddd02178297e2f55d5cd3f503586bf6165a2d7b,NaN,0.0003841682509287578081256946180
631154,2021-05-24 20:58:15+00:00,12499340,0xac810ff85e9c49fb91d29eaec00202926eafacdbb93ef7eafa36700a777f26ec,96,0xa478c2975ab1ea89e8196811f51a7b7ade33eb11,0x7a250d5630b4cf539739df2c5dacb4c659f2488d,213009497135342316140,0,0,81357628687627414,0x7a250d5630b4cf539739df2c5dacb4c659f2488d,2618.187139563668989879808248,NaN
631155,2021-05-24 20:58:18+00:00,12499341,0xc5c230229c3ff088b2bacfa972e16a8c63d87d9a86b9c68f82b77f36c95f9b68,81,0xa478c2975ab1ea89e8196811f51a7b7ade33eb11,0x7a250d5630b4cf539739df2c5dacb4c659f2488d,543867147715777757952,0,0,207723103233038242,0x3328f5f2cecaf00a2443082b657cedeaf70bfaef,2618.231382311045678289352427,NaN
631156,2021-05-24 20:58:26+00:00,12499342,0xf19b00dc0d264d3c854adb6f506f3b15fd007735c86ac1cac6ec521f3c93b0d0,328,0xa478c2975ab1ea89e8196811f51a7b7ade33eb11,0xdef1c0ded9bec7f1a1670819833240f027b25eff,659102442598017921963,0,0,251728986393417772,0xdef1c0ded9bec7f1a1670819833240f027b25eff,2618.301738076089043418215978,NaN


## 2. Querying ETH/DAI pool balance on Balancer 

In [108]:
bal_contracts = {
    '0x7afe74ae3c19f070c109a38c286684256adc656c'.lower(): 'ETH/DAI'
}

bal_contracts

{'0x7afe74ae3c19f070c109a38c286684256adc656c': 'ETH/DAI'}

In [109]:
query = """
    SELECT * FROM `blockchain-etl.ethereum_balancer.view_token_balances_subset`
    WHERE address in ({contract_list})
    AND block_number >= 11174331 
    AND block_number <= 11493364
    AND balance > 0
""".format(contract_list=(','.join(["'{}'".format(k) for k in list(bal_contracts.keys())])))

In [110]:
query_job = client.query(query)
iterator = query_job.result(timeout=MY_TIMEOUT)
rows = list(iterator)
balances = pd.DataFrame(data=[list(x.values()) for x in rows], columns=list(rows[0].keys()))
balances.head()

,token_address,address,block_number,balance
0,0x6b175474e89094c44da98b954eedeac495271d0f,0x7afe74ae3c19f070c109a38c286684256adc656c,11276859,6.515861e+24
1,0x6b175474e89094c44da98b954eedeac495271d0f,0x7afe74ae3c19f070c109a38c286684256adc656c,11279146,6.567559e+24
2,0x6b175474e89094c44da98b954eedeac495271d0f,0x7afe74ae3c19f070c109a38c286684256adc656c,11279412,6.536708e+24
3,0x6b175474e89094c44da98b954eedeac495271d0f,0x7afe74ae3c19f070c109a38c286684256adc656c,11282449,6.485017e+24
4,0x6b175474e89094c44da98b954eedeac495271d0f,0x7afe74ae3c19f070c109a38c286684256adc656c,11282457,6.485017e+24


In [153]:
tokens = {
    '0x6b175474e89094c44da98b954eedeac495271d0f': 'DAI',
    '0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2': 'ETH' # actually this is WETH
}

balances['token'] = balances['token_address'].replace(tokens)

# 3. Filtering events by blocks 

In [111]:
unchanged_balance = pd.DataFrame(balances[balances['balance'] == balances['balance'].shift(1)])
unchanged_balance.head()

,token_address,address,block_number,balance
4,0x6b175474e89094c44da98b954eedeac495271d0f,0x7afe74ae3c19f070c109a38c286684256adc656c,11282457,6.485017e+24
6,0x6b175474e89094c44da98b954eedeac495271d0f,0x7afe74ae3c19f070c109a38c286684256adc656c,11284623,6.460714e+24
13,0x6b175474e89094c44da98b954eedeac495271d0f,0x7afe74ae3c19f070c109a38c286684256adc656c,11302801,6.906190e+24
19,0x6b175474e89094c44da98b954eedeac495271d0f,0x7afe74ae3c19f070c109a38c286684256adc656c,11324053,7.336414e+24
20,0x6b175474e89094c44da98b954eedeac495271d0f,0x7afe74ae3c19f070c109a38c286684256adc656c,11324116,7.336414e+24


In [113]:
uniswap = pd.concat([uniswap[~uniswap['block_number'].isin(balances['block_number'])],
                        uniswap[uniswap['block_number'].isin(unchanged_balance['block_number'])]])

uniswap

,block_number,block_timestamp,transaction_hash,transaction_index,transaction_sender,gas_used,gas_price,token_in,token_out,amount_in,amount_out,transaction_fee
0,11174331,2020-11-02 00:00:58+00:00,0xc8d0d9abfcc80cbd5973c5c378d1160a85ff1e591496a1d0f3b5d4d321279e24,52,0x7a250d5630b4cf539739df2c5dacb4c659f2488d,109247,100500000000,DAI,ETH,22756348999999997083648,57978263943916472260,10979323500000000
1,11174335,2020-11-02 00:01:49+00:00,0xe00fdb07eca1402994be16cb246cd6e6521603696d72c6fe68b4d74c0d51507c,293,0x7a250d5630b4cf539739df2c5dacb4c659f2488d,109242,33137500000,DAI,ETH,49384474513737528836212,125728500000000000000,3620006775000000
2,11174338,2020-11-02 00:02:24+00:00,0x0312f5c9fdff0e6d4c5bcfa094dc8d5376ffdadd79861df6918d4b0d66190f58,191,0x7a250d5630b4cf539739df2c5dacb4c659f2488d,109242,34375000000,DAI,ETH,98743436648724119725195,251129300000000016384,3755193750000000
3,11174342,2020-11-02 00:03:32+00:00,0x1011a29b9d1f32e1c7fb390ef3aa990deba018c8367ad946259e2c2fdf3c1c65,335,0x7a250d5630b4cf539739df2c5dacb4c659f2488d,175233,24200000000,DAI,ETH,822119303770518643856,2088720070747335950,4240638600000000
4,11174356,2020-11-02 00:05:22+00:00,0xa078dcf23beb9382dcb695939b27c848b4f66ed95c3b5c1e4d9b5a51b100201f,391,0x7a250d5630b4cf539739df2c5dacb4c659f2488d,106239,24200000000,DAI,ETH,10001358979888216018,25406845000269931,2570983800000000
...,...,...,...,...,...,...,...,...,...,...,...,...
44048,11493279,2020-12-20 23:42:49+00:00,0x41bcfa1475c6f1dc94c3a52a7139fb7e387e6f4c7646f8ad546de34991708cca,151,0x7a250d5630b4cf539739df2c5dacb4c659f2488d,225510,55000000000,DAI,ETH,1060601747430569557622,1661187890463499895,12403050000000000
44053,11493306,2020-12-20 23:46:52+00:00,0xc5a1e9277f3340ff7f4d1be7aeb4f0e51a1badc79847324f7cea962733056b03,166,0x7a250d5630b4cf539739df2c5dacb4c659f2488d,121251,24420000000,DAI,ETH,95000000000000000000,148912330149397065,2960949420000000
44055,11493314,2020-12-20 23:48:13+00:00,0xbe19b9b00699dc4cc70adadc5470c9af7c03e44e8ed510349d728b35db87ba6d,38,0x7a250d5630b4cf539739df2c5dacb4c659f2488d,187904,24000000000,DAI,ETH,631726190435400565627,989485465064961581,4509696000000000
44056,11493314,2020-12-20 23:48:13+00:00,0x02acd760e8fb7cacd92769583769ab81c98ba9e684d82d67baac7f549eb97b5e,216,0x7a250d5630b4cf539739df2c5dacb4c659f2488d,163537,23100000000,DAI,ETH,64810422869835484326,101506843815910686,3777704700000000


# Simulating swap on Balancer

In [148]:
def calc_out_given_in(
        token_amount_in: Decimal,
        token_balance_in: Decimal,
        token_weight_in: Decimal,
        token_balance_out: Decimal,
        token_weight_out: Decimal,
        swap_fee: Decimal) -> Decimal:
    weight_ratio = token_weight_in / token_weight_out
    adjusted_in = token_amount_in * (1 - swap_fee)
    y = token_balance_in / (token_balance_in + adjusted_in)
    foo = pow(y, weight_ratio)
    bar = 1 - foo
    token_amount_out = token_balance_out * bar
    return token_amount_out

def calc_bal_amount_out(swap):
    swap_fee = Decimal(0.15)
    token_weight = Decimal(50)
    # token_amount_out = calc_out_given_in() -- TBU
    # return token_amount_out
    return swap['amount_in']
    
uniswap['bal_amount_out'] = uniswap.apply(lambda swap: calc_bal_amount_out(swap), axis=1)

In [149]:
uniswap

,block_number,block_timestamp,transaction_hash,transaction_index,transaction_sender,gas_used,gas_price,token_in,token_out,amount_in,amount_out,transaction_fee,bal_amount_out
0,11174331,2020-11-02 00:00:58+00:00,0xc8d0d9abfcc80cbd5973c5c378d1160a85ff1e591496a1d0f3b5d4d321279e24,52,0x7a250d5630b4cf539739df2c5dacb4c659f2488d,109247,100500000000,DAI,ETH,22756348999999997083648,57978263943916472260,10979323500000000,22756348999999997083648
1,11174335,2020-11-02 00:01:49+00:00,0xe00fdb07eca1402994be16cb246cd6e6521603696d72c6fe68b4d74c0d51507c,293,0x7a250d5630b4cf539739df2c5dacb4c659f2488d,109242,33137500000,DAI,ETH,49384474513737528836212,125728500000000000000,3620006775000000,49384474513737528836212
2,11174338,2020-11-02 00:02:24+00:00,0x0312f5c9fdff0e6d4c5bcfa094dc8d5376ffdadd79861df6918d4b0d66190f58,191,0x7a250d5630b4cf539739df2c5dacb4c659f2488d,109242,34375000000,DAI,ETH,98743436648724119725195,251129300000000016384,3755193750000000,98743436648724119725195
3,11174342,2020-11-02 00:03:32+00:00,0x1011a29b9d1f32e1c7fb390ef3aa990deba018c8367ad946259e2c2fdf3c1c65,335,0x7a250d5630b4cf539739df2c5dacb4c659f2488d,175233,24200000000,DAI,ETH,822119303770518643856,2088720070747335950,4240638600000000,822119303770518643856
4,11174356,2020-11-02 00:05:22+00:00,0xa078dcf23beb9382dcb695939b27c848b4f66ed95c3b5c1e4d9b5a51b100201f,391,0x7a250d5630b4cf539739df2c5dacb4c659f2488d,106239,24200000000,DAI,ETH,10001358979888216018,25406845000269931,2570983800000000,10001358979888216018
...,...,...,...,...,...,...,...,...,...,...,...,...,...
44048,11493279,2020-12-20 23:42:49+00:00,0x41bcfa1475c6f1dc94c3a52a7139fb7e387e6f4c7646f8ad546de34991708cca,151,0x7a250d5630b4cf539739df2c5dacb4c659f2488d,225510,55000000000,DAI,ETH,1060601747430569557622,1661187890463499895,12403050000000000,1060601747430569557622
44053,11493306,2020-12-20 23:46:52+00:00,0xc5a1e9277f3340ff7f4d1be7aeb4f0e51a1badc79847324f7cea962733056b03,166,0x7a250d5630b4cf539739df2c5dacb4c659f2488d,121251,24420000000,DAI,ETH,95000000000000000000,148912330149397065,2960949420000000,95000000000000000000
44055,11493314,2020-12-20 23:48:13+00:00,0xbe19b9b00699dc4cc70adadc5470c9af7c03e44e8ed510349d728b35db87ba6d,38,0x7a250d5630b4cf539739df2c5dacb4c659f2488d,187904,24000000000,DAI,ETH,631726190435400565627,989485465064961581,4509696000000000,631726190435400565627
44056,11493314,2020-12-20 23:48:13+00:00,0x02acd760e8fb7cacd92769583769ab81c98ba9e684d82d67baac7f549eb97b5e,216,0x7a250d5630b4cf539739df2c5dacb4c659f2488d,163537,23100000000,DAI,ETH,64810422869835484326,101506843815910686,3777704700000000,64810422869835484326
